In [8]:
import os
import re
import json
import numpy as np
import pandas as pd
import plotly.express as px

from itertools import chain
from unidecode import unidecode

In [9]:
proc_recursos_caminho = '../data/proc/universidade_federal_do_rio_grande_do_norte_-_ufrn'
raw_recursos_caminho = '../data/raw/universidade_federal_do_rio_grande_do_norte_-_ufrn'

In [10]:
resource_re_pattern = lambda pattern: [os.path.join(raw_recursos_caminho,filename) for filename in os.listdir(raw_recursos_caminho) if len(re.findall(pattern,filename))]

In [11]:
read_csv_pattern = lambda pattern : pd.concat(pd.read_csv(filepath,sep=';') for filepath in resource_re_pattern(pattern) if filepath.endswith('.csv'))

In [12]:
estruturas_curriculares = read_csv_pattern('estruturas-curriculares')
componentes_curriculares = read_csv_pattern('componentes-curriculares')
curriculo_componente = read_csv_pattern('curriculo-componente')

In [13]:
estruturas_curriculares = estruturas_curriculares[~estruturas_curriculares['nome_matriz'].isna()]
curriculo_componente = curriculo_componente[curriculo_componente['id_curriculo'].isin(estruturas_curriculares['id_curriculo'])]
componentes_curriculares = componentes_curriculares[componentes_curriculares['id_componente'].isin(curriculo_componente['id_componente_curricular'])]

In [14]:
filter_ec = lambda pattern : estruturas_curriculares[estruturas_curriculares['nome_curso'].apply(lambda n: len(re.findall(pattern,unidecode(n.lower())))>0 if type(n) == str else False)]

https://en.wikipedia.org/wiki/Recursive_descent_parser

In [15]:
from dash import Dash, dash_table,html,dcc,Output,Input,callback,State
import plotly.graph_objects as go

In [17]:
app = Dash()

app.layout = html.Div([
        dcc.Input(id='input_estrutura_curricular_nome',value='',type='search',style={
            'font-size' : 40
        }),
        html.Div(id='div1'),
        html.Div(id='div2',children=[
            dash_table.DataTable(
                data=estruturas_curriculares.to_dict('records'),
                columns=[
                    {"name": i, "id": i,"selectable": True} for i in estruturas_curriculares.columns
                ],
                id='datatable_estrutura_curricular',
                page_action="native",
                page_current=0,
                page_size=5,
                editable=True,
                sort_action="native",
                sort_mode="multi",
                row_selectable="single",
                selected_columns=[],
                selected_rows=[],
            )
        ]),
        html.Hr(),
        html.Div(id='div3',children=[
            dash_table.DataTable(
                data=pd.DataFrame().to_dict('records'),
                id='datatable_componente_curricular',
                page_action="native",
                page_current=0,
                page_size=15,
                editable=True,
                sort_action="native",
                sort_mode="multi",
                row_selectable="single",
                selected_columns=[],
                selected_rows=[],
            )
        ]),
        html.Hr(),
        dcc.Graph(id='graph_grades_history'),
        html.Hr(),
        html.Div(id='test'),
    ],style={
        'color':'red'
    })

@callback(
    Output('datatable_estrutura_curricular','data'),
    Input('input_estrutura_curricular_nome','value')
)
def update_table(pattern):

    id_nome_matriz = estruturas_curriculares['nome_matriz'].apply(lambda n: len(re.findall(unidecode(pattern),unidecode(n.lower())))>0 if type(n) == str else False)
    id_nome_curso  = estruturas_curriculares['nome_curso'].apply(lambda n: len(re.findall(unidecode(pattern),unidecode(n.lower())))>0 if type(n) == str else False)
    id_ec = id_nome_curso | id_nome_matriz
    return estruturas_curriculares[id_ec].to_dict('records')


@callback(
    Output('datatable_estrutura_curricular', 'style_data_conditional'),
    Output('datatable_componente_curricular','data'),
    Input('datatable_estrutura_curricular', 'selected_rows'),
    State('datatable_estrutura_curricular', 'data'),
    prevent_initial_call=True,
)
def update_styles(selected_rows, data):
    # change formatting of selected rows
    row_highlighting = [
        {
            'if': {'row_index': i},
            'background_color': '#D2F3FF'
        } for i in selected_rows
    ]

    id_cc = curriculo_componente['id_curriculo'] == data[selected_rows[0]]['id_curriculo']

    df = curriculo_componente[id_cc].merge(
        componentes_curriculares,
        how='left',
        left_on='id_componente_curricular',
        right_on='id_componente'
    )[[
        'codigo',
        'nome',
        'semestre_oferta',
        'nivel_ensino',
        'tipo_vinculo_componente',
        'ch_total',
        'modalidade',
        'id_componente'
    ]].sort_values(['tipo_vinculo_componente','semestre_oferta'])
    return row_highlighting, df.to_dict('records')



@callback(
    Output('datatable_componente_curricular', 'style_data_conditional'),
    Output('graph_grades_history','figure'),
    Output('test','children'),
    Input('datatable_componente_curricular', 'selected_rows'),
    State('datatable_componente_curricular', 'data'),
    prevent_initial_call=True,
)
def update_styles(selected_rows, data):
    row_highlighting = [
        {
            'if': {'row_index': i},
            'background_color': '#D2F3FF'
        } for i in selected_rows
    ]

    proc_recursos_caminho = '../data/proc/universidade_federal_do_rio_grande_do_norte_-_ufrn'
    proc_componentes_caminho = os.path.join(proc_recursos_caminho,'componentes')

    componente_codigo = data[selected_rows[0]]['codigo']
    componente_id = data[selected_rows[0]]['id_componente']
    
    # component_folder = os.path.join()

    return row_highlighting, go.Figure(), ['ok']

app.run(debug=True)

In [20]:
proc_recursos_caminho = '../data/proc/universidade_federal_do_rio_grande_do_norte_-_ufrn'
proc_componentes_caminho = os.path.join(proc_recursos_caminho,'componentes')
x = [os.path.join(proc_componentes_caminho,f) for f in os.listdir(os.path.join(proc_componentes_caminho)) if 'TGP0003' in f][0]
y = [os.path.join(x,f) for f in os.listdir(x) if not 'info' in f]

In [23]:
componentes_codigo,componentes_id = list(zip(*[(f.split('_')[0],int(f.split('_')[1])) for f in os.listdir(proc_componentes_caminho) if len(f.split('_'))]))

In [24]:
componentes_has_info    = [os.path.exists(os.path.join(proc_componentes_caminho,f,'info.json')) for f in os.listdir(proc_componentes_caminho)]
componentes_info_path   = [os.path.join(proc_componentes_caminho,f,'info.json') for f in os.listdir(proc_componentes_caminho)]
componentes_class_paths = [[os.path.join(proc_componentes_caminho,d,f) for f in sorted(os.listdir(os.path.join(proc_componentes_caminho,d))) if not 'info' in f] for d in os.listdir(proc_componentes_caminho)]
componentes_class_size  = [[os.path.getsize(b) for b in a] for a in componentes_class_paths]

In [25]:
df_componentes_caminhos = pd.DataFrame({
    'codigo'        : componentes_codigo,
    'id_componente' : componentes_id,
    'has_info'      : componentes_has_info,
    'info_path'     : componentes_info_path,
    'turmas'        : componentes_class_paths,
    'turmas_size'   : componentes_class_size,
    'n_turmas'      : list(map(len,componentes_class_paths))
})

In [26]:
df_componentes_caminhos = df_componentes_caminhos[df_componentes_caminhos['turmas'].apply(len)>0]
df_componentes_caminhos = df_componentes_caminhos[df_componentes_caminhos['has_info']]
df_componentes_caminhos = df_componentes_caminhos.sort_values('n_turmas',ascending=False)

In [27]:
sum(list(chain(*df_componentes_caminhos['turmas_size'].values)))/1024/1024/1024

4.692488092929125

In [ ]:
componente = df_componentes_caminhos.iloc[0]
componente

codigo                                                     ECT2104
id_componente                                                57600
has_info                                                      True
info_path        ../data/proc/universidade_federal_do_rio_grand...
turmas           [../data/proc/universidade_federal_do_rio_gran...
turmas_size      [62950, 47861, 37269, 55174, 47366, 33449, 331...
n_turmas                                                       474
Name: 1338, dtype: object

In [ ]:
with open(componente['info_path'],'r') as f:
    info = json.load(f)

In [ ]:
info

{'id_componente': 57600,
 'tipo_componente': 'MÓDULO',
 'codigo': 'ECT2104',
 'nivel': 'G',
 'nome': 'QUÍMICA GERAL',
 'unidade_responsavel': 'ESCOLA DE CIÊNCIAS E TECNOLOGIA',
 'ch_teorico': 64,
 'ch_pratico': 8,
 'ch_estagio': 0,
 'ch_total': 90,
 'ch_dedicada_docente': 90,
 'ch_ead': 18,
 'cr_max_ead': 20,
 'permite_horario_flexivel': False,
 'qtd_unidades': 3.0,
 'procedimentos_avaliacao': nan,
 'equivalencia': '( ECT1104 ) ',
 'pre_requisito': nan,
 'co_requisito': nan,
 'ementa': 'Estrutura Atômica. Periodicidade Química. Ligações Químicas. Estequiometria. Energia Química e Eletroquímica.\nFunções Orgânicas.',
 'bibliografia': nan,
 'objetivos': 'A totalização da carga horária neste módulo implicará na formação sólida do egresso nos seguintes aspectos:\r\na) Composição química e de como as unidades que constituem as substâncias estão arranjadas e como interagem entre si, determinando o elenco de propriedades que se manifestam macroscopicamente;\r\nb) Estrutura atômica e sua relaç

In [ ]:
filepaths = componente['turmas']

df_componente_turma     = []
df_componente_matricula = []

for json_filepath in filepaths:
    with open(json_filepath,'r') as json_file:
        json_content = json.load(json_file)
    json_df_matriculas = pd.DataFrame(json_content['matriculas'])
    json_df_matriculas['id_turma'] = json_content['id_turma']
    json_df_matriculas = json_df_matriculas.loc[:,~json_df_matriculas.columns.isin(['faltas_unidade'])]
    del json_content['matriculas']
    json_df_turma = pd.DataFrame(json_content)
    df_componente_matricula.append(json_df_matriculas)
    df_componente_turma.append(json_df_turma)

df_componente_turma = pd.concat(df_componente_turma)
df_componente_matricula = pd.concat(df_componente_matricula)

df_componente = df_componente_matricula.merge(df_componente_turma[['id_turma','ano','periodo']],how='left',on='id_turma')

for column in ['numero_total_faltas','media_final','id_curso']:
    df_componente[column] = df_componente[column].apply(lambda e: np.unique(e)[0])

df_componente = df_componente.explode(['unidade','nota','reposicao']).drop_duplicates()

In [ ]:
df_componente

,id_curso,discente,descricao,unidade,nota,reposicao,media_final,numero_total_faltas,id_turma,ano,periodo
0,10320810.0,00aba270e27206a0e2afbbd287a86f70,EXCLUIDA,NaN,NaN,NaN,NaN,NaN,1242252,2015,1
1,10320810.0,00aba270e27206a0e2afbbd287a86f70,INDEFERIDO,NaN,NaN,NaN,NaN,NaN,1242252,2015,1
2,10320810.0,026338ca9c3f52443ef64ce46eb2e15c,EXCLUIDA,NaN,NaN,NaN,NaN,NaN,1242252,2015,1
3,10320810.0,026338ca9c3f52443ef64ce46eb2e15c,INDEFERIDO,NaN,NaN,NaN,NaN,NaN,1242252,2015,1
4,10320810.0,05804cb20349794fcecd390bb800e74b,EXCLUIDA,NaN,NaN,NaN,NaN,NaN,1242252,2015,1
...,...,...,...,...,...,...,...,...,...,...,...
27664,10320810.0,f72ee2b3b1ee78b15f191eb379d58e78,REPROVADO,2.0,0.2,False,1.2,0.0,57704010,2022,2
27664,10320810.0,f72ee2b3b1ee78b15f191eb379d58e78,REPROVADO,3.0,0.0,False,1.2,0.0,57704010,2022,2
27665,10320810.0,faddb5b9d1a006ddff4cca363aa7aa48,APROVADO,1.0,5.8,False,7.7,0.0,57704010,2022,2
27665,10320810.0,faddb5b9d1a006ddff4cca363aa7aa48,APROVADO,2.0,7.6,False,7.7,0.0,57704010,2022,2
